In [1]:
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc.client as client
import datetime


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)
proxy_server = SimpleXMLRPCServer(("127.0.0.1", 8010), requestHandler=RequestHandler)
server = client.ServerProxy("http://127.0.0.1:8008")
stats_server = client.ServerProxy("http://127.0.0.1:8018")
stats2_server = client.ServerProxy("http://127.0.0.1:8019")


# Приём команд от клиента и передача их на сервер, а затем возврат результата
def execute_function(func_name, *params):
    try:
        work_time = datetime.datetime.now()
        result = eval('server.' + func_name)(*params)
        work_time = datetime.datetime.now() - work_time
        add_log(func_name, work_time.total_seconds(), 1)
        add_log(func_name, work_time.total_seconds(), 2)
        return result
    except:
        return 'Server is not available'
proxy_server.register_function(execute_function, 'execute')

# Добавление в лог через сервер
def add_log(func_name, work_time, server_number):
    s = stats_server if server_number == 1 else stats2_server
    try:
        s.add_log(func_name, work_time)
        return True
    finally:
        return False

# Получение данных из лога
def get_logs_slice(p_type, s_time, e_time, max_work_time, server_number):
    s = stats_server if server_number == 1 else stats2_server
    try:
        return s.get_logs_slice(p_type, s_time, e_time, max_work_time)
    except:
        return 'Log server is not available'
proxy_server.register_function(get_logs_slice, 'get_logs_slice')

print ("Listening on port 8010...")
proxy_server.serve_forever()

Listening on port 8010...


127.0.0.1 - - [19/Dec/2024 19:37:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2024 19:37:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2024 19:40:30] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2024 19:40:30] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2024 19:46:09] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2024 19:46:09] "POST /RPC2 HTTP/1.1" 200 -


KeyboardInterrupt: 